In [1]:
import numpy
import os
import tensorflow as tf
import sys
from time import time
GPU = '0'
os.environ["CUDA_VISIBLE_DEVICES"]=  GPU 
from tensorflow.examples.tutorials.mnist import input_data
def weight_variable(shape, std=0.1):
    initial = tf.truncated_normal(shape, stddev=std)
#    initial = tf.ones(shape)

    return tf.Variable(initial)

def bias_variable(shape, initVal=0.1):
    initial = tf.constant(initVal, shape=shape)
    return tf.Variable(initial)
  
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def pool_2x2(x):
    return tf.nn.avg_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')  


In [2]:
sess = tf.InteractiveSession()

In [3]:
# =========================== HF approach to solve MNIST ======================

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import numpy as np

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

if True:
    NF = 32
    NF2 = 64
    FC1 = 1024     
else:
    NF = 8
    NF2=4
    FC1 = 32
    
W_conv1 = weight_variable([5, 5, 1, NF])
b_conv1 = bias_variable([NF])
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = pool_2x2(h_conv1)
W_conv2 = weight_variable([5, 5, NF, NF2])
b_conv2 = bias_variable([NF2])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = pool_2x2(h_conv2)
W_fc1 = weight_variable([7 * 7 * NF2, FC1])
b_fc1 = bias_variable([FC1])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * NF2])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        # keep_prob = tf.placeholder(tf.float32)
        # h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([FC1, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2
params = [W_conv1, b_conv1, W_conv2, b_conv2, W_fc1, b_fc1, W_fc2, b_fc2]
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
from tensorflow.python.ops.gradients import gradients
from tensorflow.python.ops.gradients_impl import _hessian_vector_product as Hv

 

allParam = [W_conv1,b_conv1,W_conv2,b_conv2,W_fc1,b_fc1,W_fc2,b_fc2]    
NP = len(allParam)    
PH = tf.placeholder(tf.float32)
SCALE = tf.placeholder(tf.float32)

step_size=0.01
train_step = tf.train.AdamOptimizer(step_size).minimize(cross_entropy)

def computeAccuracies():
    testSize = 10000
    tBS = 100
    test_accuracy=0.0
    baches = testSize / tBS
    for i in range(baches):
        batch = mnist.test.next_batch(tBS)
        test_accuracy +=(1/(baches+0.0)) * accuracy.eval(feed_dict={ x: batch[0], y_: batch[1] })
    print "TEST:",test_accuracy

    trainSize = 60000
    tBS = 100
    train_accuracy=0.0
    baches = trainSize / tBS
    for i in range(baches):
        batch = mnist.train.next_batch(tBS)
        train_accuracy +=(1/(baches+0.0)) * accuracy.eval(feed_dict={ x: batch[0], y_: batch[1] })
    print "TRAIN",train_accuracy


In [5]:
from time import time
dx = [ tf.Variable(tf.zeros(v.get_shape())) for v in allParam ]
r = [ tf.Variable(tf.zeros(v.get_shape())) for v in allParam ]
p = [ tf.Variable(tf.zeros(v.get_shape())) for v in allParam ]
Htimesp = [ tf.Variable(tf.zeros(v.get_shape())) for v in allParam ]
momentum = [ tf.Variable(tf.zeros(v.get_shape())) for v in allParam ]


SNAPSHOT = [ tf.Variable(tf.zeros(v.get_shape())) for v in allParam ]
sess.run(tf.global_variables_initializer())

def SAVE_TO_SNAPSHOT():
    for i in xrange(NP):
        sess.run( SNAPSHOT[i].assign(allParam[i])  )

def LOAD_FROM_SNAPSHOT():
    for i in xrange(NP):
        sess.run( allParam[i].assign(SNAPSHOT[i])  )


def setDXTozeroOLD():
    for i in xrange(NP):
        print dx[i]
        dx[i]=0*dx[i] 
        sess.run(dx[i].op)
        print dx[i]

zerosResetVal = []
for i in xrange(NP):
    zerosResetVal+= [ tf.zeros(dx[i].get_shape() ) ]
reseterDX = []
reseterR = []
reseterP = []
seterRToP = []
for i in xrange(NP):
    reseterDX+=[dx[i].assign(zerosResetVal[i])]
    reseterR+=[r[i].assign(zerosResetVal[i])]
    reseterP+=[p[i].assign(zerosResetVal[i])]    
    seterRToP+=[p[i].assign(r[i])]
def setDXTozero():
    for i in xrange(NP):
        sess.run(reseterDX[i].op)
def setPTozero():
    for i in xrange(NP):
        sess.run(reseterP[i].op)
def setRTozero():
    for i in xrange(NP):
        sess.run(reseterR[i].op)
def setRToP():
    for i in xrange(NP):
        sess.run(seterRToP[i].op)
        
s=time()        
setDXTozero()        
setRTozero()        
setPTozero()  
setRToP()
print "Reseters Init took", time()-s

Reseters Init took 0.242404937744


In [6]:
PHs=[]
GeneralPH = tf.placeholder(tf.float32)
for i in xrange(NP):
    PHs+=[tf.placeholder(tf.float32)]

    


BS = 100
#BS=32
GBS= 2
HBS = 2

BS=250



NEGGRAD = gradients(tf.negative(SCALE*cross_entropy), allParam)    
Hp = Hv(cross_entropy, allParam, PHs)



HpNEW = Hv(SCALE*cross_entropy, allParam, p)    
SetHPtoHp=[]
for i in xrange(NP):
    SetHPtoHp+=[  Htimesp[i].assign(HpNEW[i]) ]
AddHPtoHp=[]
for i in xrange(NP):
    AddHPtoHp+=[  Htimesp[i].assign_add(HpNEW[i]) ]


In [7]:
s=time()
neggradient = sess.run( NEGGRAD ,feed_dict={SCALE: 1.0,x: mnist.train.images[0:BS], y_: mnist.train.labels[0:BS]})
AddGradToROP=[]
for i in xrange(NP):
    AddGradToROP+=[ r[i].assign_add(GeneralPH)  ]
def addGradientToR(gradient):
    for i in xrange(NP):
        sess.run(AddGradToROP[i].op, feed_dict={GeneralPH: gradient[i] })
setRTozero()    
addGradientToR(neggradient)
setRTozero()    
r2OP = tf.add_n([tf.reduce_sum(tf.multiply(v, v)) for v in r])
r2V = sess.run(r2OP)    
print "Init r2, adder to R took", time()-s

Init r2, adder to R took 0.945224046707


In [8]:
pHp = tf.add_n([tf.reduce_sum(tf.multiply(p[l], Htimesp[l])) for l in xrange(NP)])

In [9]:
alphaPH = tf.placeholder(tf.float32)
betaPH = tf.placeholder(tf.float32)
addAlphaPToDX=[]
subAlphaHPTor=[]
setBetaPplusRToP=[]
for i in xrange(NP):
    addAlphaPToDX+=[  dx[i].assign_add(alphaPH*p[i]) ]    
    subAlphaHPTor+=[ r[i].assign_sub(alphaPH*Htimesp[i]) ]
    setBetaPplusRToP+=[ p[i].assign_add(r[i]+betaPH*p[i]) ]
SAVE_TO_SNAPSHOT()

lrPH = tf.placeholder(tf.float32)
addDXtoParams=[]
for i in xrange(NP):
    addDXtoParams+=[  allParam[i].assign_add(lrPH*dx[i]) ]    
addPtoParams=[]
for i in xrange(NP):
    addPtoParams+=[  allParam[i].assign_add(lrPH*p[i]) ]    
normOfGradSQOP = tf.add_n([tf.reduce_sum(tf.multiply(NEGGRAD[l], NEGGRAD[l])) for l in xrange(NP)])

In [10]:
allSamples = np.arange(50000)

def RUN_CG_ALGORITHM(loss, CGSTEPS = 100):
    setDXTozero() 
    setRTozero()    
    scaleGrad = 1/(GBS+0.0)
    for i in xrange(GBS):
        neggradient = sess.run( NEGGRAD ,feed_dict={SCALE: scaleGrad,
                                                    x: mnist.train.images[allSamples[i*BS:(i+1)*BS]], 
                                                              y_: mnist.train.labels[allSamples[i*BS:(i+1)*BS]]})
        addGradientToR(neggradient)
    setRToP()
    
    scaleHess = 1/(HBS+0.0)    
    for it in xrange(CGSTEPS):
        mapDic = { SCALE:scaleHess, 
                    x: mnist.train.images[allSamples[0:BS]], 
                  y_: mnist.train.labels[allSamples[0:BS]] }
        sess.run([tmp.op for tmp in SetHPtoHp], feed_dict = mapDic)
        for j in xrange(HBS-1):
            i=j+1
            mapDic = {SCALE:scaleHess,
                      x: mnist.train.images[allSamples[i*BS:(i+1)*BS]], 
                    y_: mnist.train.labels[allSamples[i*BS:(i+1)*BS]]}
            sess.run([tmp.op for tmp in AddHPtoHp], feed_dict = mapDic)
            
            
        r2V = sess.run(r2OP)
        phpVal = sess.run(pHp)
        alphaV = r2V / phpVal
        if alphaV < 0:
            return -it
        sess.run([ [addAlphaPToDX[i].op,subAlphaHPTor[i].op] for i in xrange(NP)], feed_dict = {alphaPH:alphaV})
        r2newV = sess.run(r2OP)
        if r2newV < 10**(-10):
            break
        beta = r2newV / r2V
#         print "alpha",alphaV , 'r2',r2V, 'beta:',beta        
        sess.run([tmp.op for tmp in setBetaPplusRToP], feed_dict = {betaPH: beta} )
    return it

In [11]:
momentumLRPH = tf.placeholder(tf.float32)

addToMomentumOP=[]
for i in xrange(NP):
    addToMomentumOP+= [ momentum[i].assign(0.9*momentum[i] +0.1* momentumLRPH*dx[i]    )    ] 

addMomentumToParamOP=[]
for i in xrange(NP):
    addMomentumToParamOP+= [ allParam[i].assign_add(momentumLRPH*momentum[i]    )    ] 
    


In [12]:
LOAD_FROM_SNAPSHOT()
GBS= 2
HBS = 2
numpy.random.seed(0)
allSamples = np.arange(50000)
np.random.shuffle(allSamples)
def computeObjValue():
    f0 = 0
    for i in xrange(GBS):
        f0=f0+1/(0.0+GBS)*sess.run(cross_entropy, feed_dict={x: mnist.train.images[allSamples[i*BS:(i+1)*BS]], 
                                                         y_: mnist.train.labels[allSamples[i*BS:(i+1)*BS]] })
    return f0

In [ ]:

f0=computeObjValue()
print "F0:",f0
CGIT = 2
INC = 0
for IT in xrange(1000):
    mom = 0
    DOMOMENTUM = False
    EffectiveLRMOM = 0.0
#     if IT > 5:
#         CGIT = CGIT +1 
    status = RUN_CG_ALGORITHM(cross_entropy,CGIT)
    if status > 0:
        action = addDXtoParams
        LR = 1.0
        DOMOMENTUM = True
#         DOMOMENTUM = False
    else:
        action = addPtoParams
        LR = 1.0
    EffectiveLR = 0.0    
    NOTDONE = True
    notdoneIT = 0
    while NOTDONE:
#        print "effecrt",notdoneIT, EffectiveLR
        EffectiveLR+=LR
        sess.run([tmp.op for tmp in action], feed_dict = {lrPH: LR} )   
        fnew =computeObjValue()
        if fnew > f0:
            LR = -np.abs(LR)/2
            notdoneIT +=1
            if notdoneIT > 20:               
                NOTDONE = False
        else:
            NOTDONE = False
            
            
    if DOMOMENTUM:
        sess.run([tmp.op for tmp in addToMomentumOP], feed_dict = {momentumLRPH: EffectiveLR} )   
        tmpi = 0    
        NOTDONE = True         
        LRm = 32.0*2
        while NOTDONE:
            EffectiveLRMOM+=LRm
            sess.run([tmp.op for tmp in addMomentumToParamOP], feed_dict = {momentumLRPH: LRm} )   
            fnewMOM = computeObjValue()
#             print tmpi,"MOMENTUM",fnewMOM,fnew, fnewMOM-fnew
            if fnewMOM > fnew :
                LRm = -np.abs(LRm)/2
            else:
                NOTDONE = False
            tmpi+=1
            if tmpi > 20:
                NOTDONE = False
                sess.run([tmp.op for tmp in addMomentumToParamOP], feed_dict = {momentumLRPH: -EffectiveLRMOM} )   
                EffectiveLRMOM = 0
                fnewMOM =computeObjValue()
        mom = fnew-fnewMOM 
        fnew = fnewMOM 

    print IT,"Fval:", fnew, fnew-f0,   status,"LR",LR, "LRmom",EffectiveLRMOM,"MOMdec",mom, mom/(fnew-f0)
        
    if IT > 50:
        if INC == 0 and  (GBS < 50000/BS):
            HBS=HBS+1
            if HBS > 5:
                HBS = 5
            if GBS > 4:
                GBS=min(GBS*2,   50000/BS )
            else:
                GBS=GBS+1
            fnew = 1
            print "CURENT SIZE OF STUFF IS ", BS*GBS
        if INC > 50:
            INC=-1
        INC=INC+1
         
        
        
        
#    if GBS < 50000/BS:     
#        GBS += 1    
#    sizeofGradSQ = sess.run(normOfGradSQOP, feed_dict={x: mnist.train.images[0:BS], y_: mnist.train.labels[0:BS]})
#    np.random.shuffle(allSamples)
    f0=fnew

F0: 6.46151733398
0 Fval: 6.07385253906 -0.387664794922 1 LR 1.0 LRmom 0 MOMdec -7.39097595215e-06 1.90653782571e-05
1 Fval: 4.58420825005 -1.48964428902 1 LR 1.0 LRmom 0 MOMdec -8.34465026855e-06 5.60177374564e-06
2 Fval: 2.87160503864 -1.7126032114 1 LR 1.0 LRmom 0 MOMdec -1.31130218506e-06 7.6567775672e-07
3 Fval: 2.14384126663 -0.727763772011 1 LR 1.0 LRmom 4.0 MOMdec 0.0126742124557 -0.017415283562
4 Fval: 1.92318361998 -0.220657646656 1 LR 1.0 LRmom 0 MOMdec 3.57627868652e-07 -1.62073634915e-06
5 Fval: 1.66290581226 -0.260277807713 1 LR 1.0 LRmom 2.0 MOMdec 0.00475418567657 -0.0182658126652
6 Fval: 1.3744494319 -0.288456380367 1 LR -0.5 LRmom 8.0 MOMdec 0.0708504915237 -0.2456194293
7 Fval: 1.13227939606 -0.242170035839 1 LR 1.0 LRmom 16.0 MOMdec 0.0459563136101 -0.189768785601
8 Fval: 0.923631608486 -0.208647787571 1 LR 1.0 LRmom 0 MOMdec 1.19209289551e-07 -5.71342217133e-07
9 Fval: 0.69247251749 -0.231159090996 1 LR 1.0 LRmom 16.0 MOMdec 0.132791221142 -0.574458138634
10 Fval: 

79 Fval: 0.000344506202964 -2.41200323217e-05 1 LR 1.0 LRmom 8.0 MOMdec 4.43505511309e-06 -0.183874343697
80 Fval: 0.000303408553009 -4.10976499552e-05 1 LR 1.0 LRmom 32.0 MOMdec 1.70490966411e-05 -0.414843589833
81 Fval: 0.000245386933481 -5.80216195279e-05 1 LR 1.0 LRmom 8.0 MOMdec 8.48387814282e-06 -0.146219257784
82 Fval: 0.000221245058735 -2.41418747464e-05 1 LR 1.0 LRmom 64.0 MOMdec 8.02120484877e-06 -0.332252773781
83 Fval: 0.000160928878662 -6.0316180073e-05 1 LR 1.0 LRmom 8.0 MOMdec 3.78987654888e-06 -0.0628334974843
84 Fval: 0.000135439081835 -2.54897968262e-05 1 LR 1.0 LRmom 16.0 MOMdec 6.03893082977e-06 -0.236915612586
85 Fval: 0.000120206112721 -1.52329691142e-05 1 LR 1.0 LRmom 16.0 MOMdec 2.63628559575e-06 -0.173064461431
86 Fval: 0.0001073617168 -1.28443959208e-05 1 LR 1.0 LRmom 16.0 MOMdec 2.19909755591e-06 -0.171210664127
87 Fval: 9.51758120209e-05 -1.21859047795e-05 1 LR 1.0 LRmom 16.0 MOMdec 3.09359650904e-06 -0.253866788312
88 Fval: 8.39010923907e-05 -1.12747196302e

156 Fval: 0.879819905758 -0.120180094242 1 LR -0.5 LRmom 0 MOMdec -1.28269195557e-05 0.000106730816252
157 Fval: 0.226222378016 -0.653597527742 1 LR 1.0 LRmom 0 MOMdec -2.11447477338e-06 3.23513275928e-06
158 Fval: 0.140627361089 -0.0855950169265 1 LR 1.0 LRmom 0 MOMdec -2.13272869615e-07 2.49165053379e-06
159 Fval: 0.116750168987 -0.0238771921024 1 LR 1.0 LRmom 4.0 MOMdec 0.00359512602445 -0.150567370277
160 Fval: 0.0898908368777 -0.0268593321089 1 LR 1.0 LRmom 8.0 MOMdec 0.0116475375718 -0.43364956078
161 Fval: 0.0808408979326 -0.0090499389451 1 LR 1.0 LRmom 32.0 MOMdec 0.00118514369242 -0.130955987616
162 Fval: 0.0532994728535 -0.0275414250791 1 LR 1.0 LRmom 2.0 MOMdec 0.000364031456411 -0.0132175969604
163 Fval: 0.0447844559327 -0.00851501692086 1 LR 1.0 LRmom 8.0 MOMdec 0.00181164797395 -0.212759174854
164 Fval: 0.0374238655204 -0.00736059041228 1 LR 1.0 LRmom 8.0 MOMdec 0.00112468872685 -0.152798710953
165 Fval: 0.0315577199217 -0.0058661455987 1 LR 1.0 LRmom 4.0 MOMdec 0.0006170

233 Fval: 0.00194023278891 -0.000415655568941 1 LR 1.0 LRmom 0 MOMdec -9.58098098644e-09 2.3050288995e-05
234 Fval: 0.00165730910376 -0.000282923685154 1 LR 1.0 LRmom 16.0 MOMdec 8.95148667041e-05 -0.316392269016
235 Fval: 0.00149517903337 -0.000162130070385 1 LR 1.0 LRmom 8.0 MOMdec 5.62078785151e-06 -0.0346683859335
236 Fval: 0.00133626306197 -0.000158915971406 1 LR 1.0 LRmom 8.0 MOMdec 4.00471617468e-06 -0.0252002120319
237 Fval: 0.00118237560382 -0.000153887458146 1 LR 1.0 LRmom 8.0 MOMdec 2.93355667964e-05 -0.190630004224
238 Fval: 0.00108470772975 -9.76678740699e-05 1 LR 1.0 LRmom 8.0 MOMdec 1.4237704454e-05 -0.145776741734
239 Fval: 0.000944485102082 -0.00014022262767 1 LR 1.0 LRmom 16.0 MOMdec 4.40827629063e-05 -0.314376956408
240 Fval: 0.000851836707443 -9.26483946387e-05 1 LR 1.0 LRmom 4.0 MOMdec 4.76837158203e-07 -0.00514673956373
241 Fval: 0.000687087571714 -0.000164749135729 1 LR 1.0 LRmom 64.0 MOMdec 0.000119128753431 -0.723091826273
242 Fval: 0.000563255374436 -0.0001238

309 Fval: 0.000268539184617 -6.86939783918e-05 1 LR 1.0 LRmom 16.0 MOMdec 1.01447760244e-05 -0.14768071761
310 Fval: 0.000256223195902 -1.23159887153e-05 1 LR 1.0 LRmom 2.0 MOMdec 2.27853888646e-07 -0.0185006574716
311 Fval: 0.000254506974306 -1.71622159542e-06 1 LR 1.0 LRmom 2.0 MOMdec 3.73896909877e-08 -0.0217860508734
CURENT SIZE OF STUFF IS  10000
312 Fval: 0.655033504963 -0.344966495037 1 LR -0.125 LRmom 0 MOMdec -6.40749931347e-06 1.85742656335e-05
313 Fval: 0.173680904135 -0.481352600828 1 LR 1.0 LRmom 0 MOMdec -1.22659839694e-06 2.5482326154e-06
314 Fval: 0.0894586839 -0.0842222202351 1 LR 1.0 LRmom 0 MOMdec -1.2657255867e-07 1.50284044183e-06
315 Fval: 0.0724522686927 -0.0170064152073 1 LR -0.5 LRmom 1.0 MOMdec 7.56371002353e-05 -0.00444756283517
316 Fval: 0.062641806918 -0.00981046177476 1 LR -0.25 LRmom 8.0 MOMdec 0.00426767897297 -0.435013057587
317 Fval: 0.0619038646779 -0.000737942240085 1 LR 1.0 LRmom 0 MOMdec -2.82976543517e-08 3.83467063065e-05
318 Fval: 0.052155166148

386 Fval: 0.015081353116 -0.000487018877175 1 LR -0.5 LRmom 32.0 MOMdec 0.000199866318144 -0.410387209842
387 Fval: 0.0139128877403 -0.00116846537567 1 LR 1.0 LRmom 0 MOMdec -2.56783096106e-08 2.19760980045e-05
388 Fval: 0.0132938020135 -0.000619085726794 1 LR -0.5 LRmom 8.0 MOMdec 0.000201402069069 -0.325321777506
389 Fval: 0.0119731650164 -0.00132063699712 1 LR -0.5 LRmom 64.0 MOMdec 0.00109952123894 -0.832568859831
390 Fval: 0.0118389877287 -0.000134177287691 1 LR 1.0 LRmom 0 MOMdec -4.76240529738e-08 0.000354933787925
391 Fval: 0.0117755223153 -6.34654134046e-05 1 LR 1.0 LRmom 0 MOMdec -3.60014382889e-09 5.67260754442e-05
392 Fval: 0.0111882019235 -0.000587320391787 1 LR 1.0 LRmom 0.0001220703125 MOMdec 4.2593455863e-09 -7.2521670384e-06
393 Fval: 0.0111322546145 -5.59473090107e-05 1 LR 1.0 LRmom 4.0 MOMdec 2.75299797067e-05 -0.492069774106
394 Fval: 0.0107583557256 -0.000373898888938 1 LR 1.0 LRmom 0.000244140625 MOMdec 7.69796315407e-09 -2.05883552528e-05
395 Fval: 0.010434580472

462 Fval: 0.00942520266108 -0.000111383182229 1 LR -0.5 LRmom 2.0 MOMdec 7.78995163274e-06 -0.0699383109444
463 Fval: 0.0092773075834 -0.00014789507768 1 LR -0.5 LRmom 2.0 MOMdec 3.14089993486e-07 -0.00212373527512
464 Fval: 0.00917211720953 -0.00010519037387 1 LR -0.5 LRmom 8.0 MOMdec 2.90390969894e-05 -0.276062304193
465 Fval: 0.00905257976556 -0.00011953744397 1 LR -0.5 LRmom 4.0 MOMdec 9.12204195629e-06 -0.0763111678927
466 Fval: 0.00892298248364 -0.000129597281921 1 LR -0.5 LRmom 16.0 MOMdec 6.74820490531e-05 -0.520705743615
467 Fval: 0.0087923886902 -0.000130593793438 1 LR -0.5 LRmom 4.0 MOMdec 7.50028993935e-06 -0.0574322082384
CURENT SIZE OF STUFF IS  50000
468 Fval: 0.0212745938677 -0.978725406132 1 LR 1.0 LRmom 0 MOMdec -6.52199378288e-08 6.66376262639e-08
469 Fval: 0.0211923126201 -8.22812475963e-05 1 LR 1.0 LRmom 0 MOMdec -8.55611870174e-08 0.00103986253875
470 Fval: 0.0187041644426 -0.00248814817751 1 LR -0.5 LRmom 16.0 MOMdec 0.000475960450131 -0.191291039028
471 Fval: 0.

In [14]:
LOAD_FROM_SNAPSHOT()
numpy.random.seed(0)
allSamples = np.arange(50000)
np.random.shuffle(allSamples)

f0 =computeObjValue()
print "F0:",f0
for IT in xrange(200):
    for i in xrange(5000/BS):
        train_step.run(feed_dict={x: mnist.train.images[allSamples[i*BS:(i+1)*BS]],
                                  y_: mnist.train.labels[allSamples[i*BS:(i+1)*BS]] })
    np.random.shuffle(allSamples)
    fnew = computeObjValue()
#    sizeofGradSQ = sess.run(normOfGradSQOP, feed_dict={x: mnist.train.images[0:BS], y_: mnist.train.labels[0:BS]})
    print IT,"Fval:", fnew, fnew-f0 
    f0=fnew


F0: 6.62750877619
0 Fval: 2.36117739677 -4.26633137941
1 Fval: 2.30990394354 -0.0512734532356
2 Fval: 2.31481955528 0.00491561174393
3 Fval: 2.31782923341 0.00300967812538
4 Fval: 2.30906274796 -0.00876648545265
5 Fval: 2.30803745747 -0.0010252904892
6 Fval: 2.30762248039 -0.000414977073668
7 Fval: 2.30513979435 -0.00248268604279
8 Fval: 2.312105726 0.00696593165398
9 Fval: 2.30928756714 -0.00281815886497
10 Fval: 2.31654127479 0.00725370764732
11 Fval: 2.31432394385 -0.00221733093262
12 Fval: 2.31579670548 0.00147276163101
13 Fval: 2.32014364243 0.00434693694115
14 Fval: 2.30694995761 -0.0131936848164
15 Fval: 2.30871476531 0.00176480770111
16 Fval: 2.31069389224 0.00197912693024
17 Fval: 2.31715662241 0.0064627301693
18 Fval: 2.30512755394 -0.01202906847
19 Fval: 2.30753963947 0.00241208553314
20 Fval: 2.30634457231 -0.00119506716728
21 Fval: 2.30663740635 0.000292834043502
22 Fval: 2.30897816777 0.00234076142311
23 Fval: 2.31255458593 0.00357641816139
24 Fval: 2.31011479139 -0.00243

197 Fval: 2.30138063669 -0.000310585498812
198 Fval: 2.3014251411 4.450440407e-05
199 Fval: 2.30153362036 0.000108479261398
